#### Exploration of 2023 Labels 

* Data annotated by Simon Geikie July/August 2023
* Data sourced for Ordnance Survey July/August 2023

| **Key**                     | **Meaning**                                                             |
|-----------------------------|-------------------------------------------------------------------------|
| A: Present                  | Present in OS data \| Present in the 2023 AP \| Present in 2014 AP      |
| B: Absent                   | Present in OS data \| Absent in the 2023 AP \| Absent in 2014 AP        |
| C: Absent (Present in 2014) | Absent in OS data \| Absent in the 2023 AP \| Present in 2014 AP        |
| E: Obscured (Vegetation)    | Obscured by trees, scrub or long grass.                                 |
| H: Urban                    | A mask created to simplify the detection of walls to only the open landscape |

In [7]:
# load libraries 
import fiona
import shapely as sp
from shapely import LineString
import matplotlib.pyplot as plt

In [3]:
# load data
walls = fiona.open("../content/labels/Walls_Features_for_CNN.shp")

In [9]:
walls = LineString(walls)
walls.length

ValueError: linestrings: Input operand 0 does not have enough dimensions (has 1, gufunc core with signature (i, d)->() requires 2)